In [2]:
import requests
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from datetime import datetime
import matplotlib.pyplot as plt

# Replace 'your_api_key' with your actual Token Terminal API key
API_KEY = '2acb79dd-f8ca-48b5-b10c-bbee2290ed94'
BASE_URL = 'https://api.tokenterminal.com/v2'


In [3]:
def fetch_projects():
    url = f'{BASE_URL}/projects'
    headers = {
        'Authorization': f'Bearer {API_KEY}'
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for HTTP errors
    return response.json()

def fetch_metrics():
    url = f'{BASE_URL}/metrics'
    headers = {
        'Authorization': f'Bearer {API_KEY}'
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for HTTP errors
    return response.json()

def fetch_time_series_data(metric_ids, project_ids, start_date, end_date):
    cache = {project_id: [] for project_id in project_ids}
    project_ids_str = ','.join(project_ids)
    
    for metric_id in metric_ids:
        url = f'{BASE_URL}/metrics/{metric_id}'
        headers = {
            'Authorization': f'Bearer {API_KEY}'
        }
        params = {
            'project_ids': project_ids_str,
            'start': start_date,
            'end': end_date
        }
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 404:
            print(f"Data for projects or metric ID '{metric_id}' not found.")
            continue
        response.raise_for_status()  # Raise an exception for other HTTP errors
        data = response.json()['data']
        
        for entry in data:
            cache[entry['project_id']].append(entry)
    
    for project_id in cache:
        if cache[project_id]:
            df = pd.DataFrame(cache[project_id])
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            df = df.pivot(index='timestamp', columns='metric_id', values='value')
            df.reset_index(inplace=True)
            if 'token_supply_circulating' in df.columns and 'token_supply_maximum' in df.columns:
                df['circulating_supply_ratio'] = df['token_supply_circulating'] / df['token_supply_maximum']
            cache[project_id] = df
        else:
            cache[project_id] = pd.DataFrame()
    
    return cache


In [4]:
def plot_time_series(data_dict, project_id):
    fig = go.Figure()
    data = data_dict[project_id]
    if not data.empty:
        data.set_index('timestamp', inplace=True)
        for column in data.columns:
            fig.add_trace(go.Scatter(x=data.index, y=data[column], mode='lines+markers', name=column))
    
    fig.update_layout(
        title=f'Metrics Over Time for {project_id.capitalize()}',
        xaxis_title='Date',
        yaxis_title='Value',
        legend_title='Metric',
        template='plotly_white'
    )
    fig.show()

def plot_circulating_supply(data_dict):
    fig = go.Figure()
    
    for project_id, data in data_dict.items():
        if 'circulating_supply_ratio' in data.columns:
            data.set_index('timestamp', inplace=True)
            fig.add_trace(go.Scatter(x=data.index, y=data['circulating_supply_ratio'], mode='lines+markers', name=project_id))
            
            # Calculate trendline
            x = np.arange(len(data.index))
            y = data['circulating_supply_ratio'].values
            if len(x) > 1:  # Ensure there's enough data to fit a line
                trendline = np.polyfit(x, y, 1)
                trend = np.poly1d(trendline)
                trend_x = data.index
                trend_y = trend(x)
                fig.add_trace(go.Scatter(x=trend_x, y=trend_y, mode='lines', name=f'{project_id} Trend', line=dict(dash='dash')))
                
                # Print the trendline equation
                print(f'Trendline equation for {project_id}: y = {trendline[0]:.2f}x + {trendline[1]:.2f}')
    
    fig.update_layout(
        title='Circulating Supply Ratio Over Time for Selected Projects',
        xaxis_title='Date',
        yaxis_title='Circulating Supply Ratio',
        legend_title='Project',
        template='plotly_white'
    )
    fig.show()


In [5]:
projects = fetch_projects()
metrics = fetch_metrics()

# Convert projects and metrics to DataFrame for easier manipulation
projects_df = pd.DataFrame(projects['data'])
metrics_df = pd.DataFrame(metrics['data'])

metric_ids = ['price', 'token_supply_circulating', 'token_supply_maximum']

# Limit to a subset of project IDs
project_ids = ['polygon','aave','chainlink','worldcoin']  # Example project IDs
start_date = '2023-01-01'
end_date = '2023-12-31'

# Fetch time series data for the given metrics and project IDs
time_series_data = fetch_time_series_data(metric_ids, project_ids, start_date, end_date)

# Print the fetched data for inspection
print(time_series_data)

# Plot the circulating supply ratio over time for the different projects with trendlines
plot_circulating_supply(time_series_data)


{'polygon': metric_id                 timestamp     price  token_supply_circulating  \
0         2023-01-01 00:00:00+00:00  0.759338              8.959945e+09   
1         2023-01-02 00:00:00+00:00  0.781776              8.959945e+09   
2         2023-01-03 00:00:00+00:00  0.777612              8.959892e+09   
3         2023-01-04 00:00:00+00:00  0.802718              8.959892e+09   
4         2023-01-05 00:00:00+00:00  0.790964              8.959892e+09   
..                              ...       ...                       ...   
360       2023-12-27 00:00:00+00:00  1.040819              9.527214e+09   
361       2023-12-28 00:00:00+00:00  0.996759              9.527211e+09   
362       2023-12-29 00:00:00+00:00  0.968345              9.527225e+09   
363       2023-12-30 00:00:00+00:00  0.957385              9.527225e+09   
364       2023-12-31 00:00:00+00:00  0.972824              9.527222e+09   

metric_id  token_supply_maximum  circulating_supply_ratio  
0                  1.000000